## Training

![](../images/ml-artifacts-1.jpg)

In [28]:
!rm -fr train
!mkdir train

In [16]:
import warnings
import pandas as pd
import tensorflow as tf

warnings.filterwarnings('ignore')

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

BUCKET = 'train/'

# Extraction process
dataset = pd.read_csv('https://storage.googleapis.com/jchavezar-public-datasets/auto-mpg.csv')
dataset.tail()

dataset.isna().sum()
dataset = dataset.dropna()
dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})
dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')
dataset.tail()

train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

train_stats = train_dataset.describe()
train_stats.pop("MPG")
train_stats = train_stats.transpose()

train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')


def norm(x):
    return (x - train_stats['mean']) / train_stats['std']


normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)


def build_model():
    model_ai = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
    optimizer = tf.keras.optimizers.RMSprop(0.001)
    model_ai.compile(loss='mse',
                     optimizer=optimizer,
                     metrics=['mae', 'mse'])
    return model_ai


model = build_model()
model.summary()
EPOCHS = 1000

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
early_history = model.fit(normed_train_data, train_labels,
                          epochs=EPOCHS, validation_split=0.2,
                          callbacks=[early_stop])

# Export model and save to GCS
print(BUCKET)
model.save(BUCKET)

2023-02-02 13:27:03.937444: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-02 13:27:05.128385: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-02 13:27:05.128646: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-02 13:27:08.410911: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

2.11.0


2023-02-02 13:27:11.288380: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-02 13:27:11.288474: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-02 13:27:11.288577: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (penguin): /proc/driver/nvidia/version does not exist
2023-02-02 13:27:11.289846: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                640       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 4,865
Trainable params: 4,865
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
8/8 [==============================] - 1s 51ms/step - loss: 571.7780 - mae: 22.7181 - mse: 571.7780 - val_loss: 561.2507 - val_mae: 22.4582 - val_mse: 561.2507
Epoch 2/1000
8/8 [==============================] - 0s 13ms/step - loss: 519.0931 - mae: 21.5424 - mse: 519.0931 - val_loss: 509.2206 - val_mae: 

INFO:tensorflow:Assets written to: train/assets


INFO:tensorflow:Assets written to: train/assets


## Create Container Image and Save it on Google Registry

![](../images/ml-artifacts-2.jpg)

In [4]:
!rm -fr app
!mkdir app

In [13]:
!pwd

/home/atreides/predictions


In [19]:
%%writefile app/main.py
from fastapi import Request, FastAPI
from tensorflow import keras
import json
import os

app = FastAPI()

model = keras.models.load_model('.')

@app.get('/')
def get_root():
    return {'message': 'Welcome mpg API: miles per gallon prediction'}


@app.get('/health_check')
def health():
    return 200


if os.environ.get('AIP_PREDICT_ROUTE') is not None:
    method = os.environ['AIP_PREDICT_ROUTE']
else:
    method = '/predict'

print(method)
@app.post(method)
async def predict(request: Request):
    print("----------------- PREDICTING -----------------")
    body = await request.json()
    instances = body["instances"]
    outputs = model.predict(instances)
    response = outputs.tolist()
    print("----------------- OUTPUTS -----------------")

    return {"predictions": response}

Overwriting app/main.py


In [9]:
%%writefile Dockerfile
FROM tiangolo/uvicorn-gunicorn-fastapi:python3.7

COPY train /app
COPY app /app
WORKDIR /app
RUN pip install sklearn joblib pandas tensorflow
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8080"]

EXPOSE 8080

Overwriting Dockerfile


In [10]:
!gcloud builds submit -t gcr.io/jchavezar-demo/custom-local-train:v1 .

Creating temporary tarball archive of 14 file(s) totalling 393.4 KiB before compression.
Uploading tarball of [.] to [gs://jchavezar-demo_cloudbuild/source/1675355149.937054-03dbb02a05ba4e03b6de070722869b30.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/jchavezar-demo/locations/global/builds/d2c8e558-5b0d-4e35-a51f-721e1d89948e].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/d2c8e558-5b0d-4e35-a51f-721e1d89948e?project=569083142710 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "d2c8e558-5b0d-4e35-a51f-721e1d89948e"

FETCHSOURCE
Fetching storage object: gs://jchavezar-demo_cloudbuild/source/1675355149.937054-03dbb02a05ba4e03b6de070722869b30.tgz#1675355150168811
Copying gs://jchavezar-demo_cloudbuild/source/1675355149.937054-03dbb02a05ba4e03b6de070722869b30.tgz#1675355150168811...
/ [1 files][205.0 KiB/205.0 KiB]                                                
Operation completed over 1 object

## Upload Image to Model Registry

In [17]:
from google.cloud import aiplatform as aip

my_model = aip.Model.upload(
    display_name="my-model-latest", 
    serving_container_ports=[8080],
    serving_container_health_route="/health_check",
    serving_container_image_uri="gcr.io/jchavezar-demo/custom-local-train:v1")

Creating Model


INFO:google.cloud.aiplatform.models:Creating Model


Create Model backing LRO: projects/569083142710/locations/us-central1/models/8478890514664390656/operations/5970245678083866624


INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/569083142710/locations/us-central1/models/8478890514664390656/operations/5970245678083866624


Model created. Resource name: projects/569083142710/locations/us-central1/models/8478890514664390656@1


INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/569083142710/locations/us-central1/models/8478890514664390656@1


To use this Model in another session:


INFO:google.cloud.aiplatform.models:To use this Model in another session:


model = aiplatform.Model('projects/569083142710/locations/us-central1/models/8478890514664390656@1')


INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/569083142710/locations/us-central1/models/8478890514664390656@1')


## Deploy on Vertex Endpoints

![](images/ml-artifacts-3.jpg)

In [18]:
my_model.deploy(
    deployed_model_display_name='local-model',
    machine_type='n1-standard-4'
)

Creating Endpoint


INFO:google.cloud.aiplatform.models:Creating Endpoint


Create Endpoint backing LRO: projects/569083142710/locations/us-central1/endpoints/6297919041017217024/operations/284451148528615424


INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/569083142710/locations/us-central1/endpoints/6297919041017217024/operations/284451148528615424


Endpoint created. Resource name: projects/569083142710/locations/us-central1/endpoints/6297919041017217024


...

Deploy Endpoint model backing LRO: projects/569083142710/locations/us-central1/endpoints/6297919041017217024/operations/1498171248104964096


INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/569083142710/locations/us-central1/endpoints/6297919041017217024/operations/1498171248104964096


Endpoint model deployed. Resource name: projects/569083142710/locations/us-central1/endpoints/6297919041017217024


INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/569083142710/locations/us-central1/endpoints/6297919041017217024


resource name: projects/569083142710/locations/us-central1/endpoints/6297919041017217024